# Multi-Classification Machine Learning for Malware Analysis
## 9 Types of Malware in this dataset:
1. Ramnit         - RAT
2. Lollipop       - Adware
3. Kelihos_ver3   - RAT
4. Vundo          - Adware
5. Simda          - Botnet
6. Tracur         - Malicious Browser Plugin
7. Kelihos_ver1   - RAT
8. Obfuscator.ACY - Obfuscates other malware/information
9. Gatak          - RAT

## Game Plan:

- Look into creating more metrics to show off my model
- Improve the way I import data for the model
- Explain my code and solution in detail
- Port into the main program/script



## Imports

In [85]:
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install torchvision
!{sys.executable} -m pip install jupyterthemes

import sys
import os
import re
import csv
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import json
from collections import Counter, OrderedDict, defaultdict
import shutil
from pathlib import Path #Convert all directory accesses to this
from functools import reduce
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
import heapq

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Functions

In [86]:
def listFilesInDirectory(directoryContainingFiles):
    return glob.glob(directoryContainingFiles) 

def stripFilePathAndExtension(filePath, prefixToStrip, suffixToStrip):
    filePath = filePath.replace(prefixToStrip, "")
    filePath = filePath.replace(suffixToStrip, "")
    #return filePath
    return Path(filePath).stem

def replaceFilePathAndExtension(filePath, prefixToStrip, prefixToInsert, suffixToStrip, suffixToInsert):
    filePath = filePath.replace(prefixToStrip, prefixToInsert)
    filePath = filePath.replace(suffixToStrip, suffixToInsert)
    return filePath

def printDataFrame(dataframe):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(dataframe)

def zeroOutDataframe(dataframe):
    dataframe = dataframe.fillna(0)
    return dataframe

def countEntriesInDataframe(dataframe):
    return np.count_nonzero(dataframe)

def sortDictionary(dictionary):
    returnVal = sorted(dict(Counter(dictionary)).items(), key=lambda kv:
                 (kv[1], kv[0]))
    return returnVal

def fileNewlineIntoList(filePath):
    lineList = []
    with open(filePath) as openFile:
        for line in openFile:
            temp = line.strip()
            lineList.append(temp)
    return lineList

def stripNewlineAndWhitespace(textStringToStrip):
    textStringToStrip = textStringToStrip.replace("\t","")
    textStringToStrip = textStringToStrip.replace("\n","")
    textStringToStrip = textStringToStrip.replace(" ","")
    return textStringToStrip

def stripNewlineAndWhitespaceFromList(listToStrip):
    for i in range(0,len(listToStrip)):
        listToStrip[i] = listToStrip[i].replace("\t","")
        listToStrip[i] = listToStrip[i].replace("\n","")
        listToStrip[i] = listToStrip[i].replace(" ","")
    return listToStrip

def regexSearchFile(filePath, regexPattern):
    with open(filePath) as openFile:
        matches = re.findall(regexPattern, openFile.read())
    openFile.close()
    return matches

def cleanFileNameList(fileNameList,malwareClass):
    filePathToNameDict = {}
    for i in range(0, len(fileNameList)): 
        strippedFile = stripFilePathAndExtension(fileNameList[i], "/home/eddy/machine-learning/data/dataset-subset/class-"+str(malwareClass)+"/", ".asm") #FIX THIS TO ALLOW FOR DIFFERENT CLASSES
        filePathToNameDict[strippedFile] = fileNameList[i]
        fileNameList[i] = strippedFile
    return fileNameList

def generateClassDataFrame(instructionList,fileNameListForClass):
    return zeroOutDataframe(pd.DataFrame(columns=instructionList,index=fileNameListForClass))

def moveFilesToClassFolders(backupFileList, fullFileNamesListFromCSV): #Old and working before I tried the next version
#    fullFileNamesListFromCSV.set_index("Id",inplace=True)
#    for file in backupFileList: # file is the full path to the file, fileClean is just the name of the file without extension
#        fileClean = stripFilePathAndExtension(file,"/home/eddy/machine-learning/data/dataset-subsetFullInitialSubset/",".asm")
#        shutil.copyfile(file,"/home/eddy/machine-learning/data/dataset-subset/class-"+str(fullFileNamesListFromCSV.loc[fileClean,"Class"])+"/"+str(fullFileNamesListFromCSV.loc[fileClean].name)+".asm")
#        #print("from: "+file+" ------------- to: "+"/home/eddy/machine-learning/data/dataset-subset/class-"+str(fullFileNamesListFromCSV.loc[fileClean,"Class"])+"/"+str(fullFileNamesListFromCSV.loc[fileClean].name)+".asm")
#def moveFilesToClassFolders(backupFileList, fullFileNamesListFromCSV): #Experimental
    fullFileNamesListFromCSV.set_index("Id",inplace=True)
    for fileIndex in range(0,len(backupFileList)): # file is the full path to the file, fileClean is just the name of the file without extension
        fileClean = stripFilePathAndExtension(backupFileList[fileIndex],"/home/eddy/machine-learning/data/dataset-subsetFullInitialSubset/",".asm")
        try:
            shutil.copyfile(backupFileList[fileIndex],"/home/eddy/machine-learning/data/dataset-subset/class-"+str(fullFileNamesListFromCSV.loc[fileClean,"Class"])+"/"+str(fullFileNamesListFromCSV.loc[fileClean].name)+".asm")
        except:
            fileIndex = fileIndex + 1
        #print("from: "+file+" ------------- to: "+"/home/eddy/machine-learning/data/dataset-subset/class-"+str(fullFileNamesListFromCSV.loc[fileClean,"Class"])+"/"+str(fullFileNamesListFromCSV.loc[fileClean].name)+".asm")

def generateFilenameToDirectoryDict(fileDirectory):
    filePathToNameDict = {}
    for file in fileDirectory:
        filePathToNameDict[Path(file).stem] = file
    return filePathToNameDict

def populateMalwareDataframe(fileDirectoryTopLevel,instructionList):

    filePathToNameDict = generateFilenameToDirectoryDict(listFilesInDirectory(fileDirectoryTopLevel))
    dataFrame = zeroOutDataframe(pd.DataFrame(columns=instructionList,index=filePathToNameDict.keys()))

    for file in filePathToNameDict.keys(): # Go through every file in our directory
        fileDirectory = filePathToNameDict[file] # Convert using dict here
        instructionsForThisFile = stripNewlineAndWhitespaceFromList(regexSearchFile(fileDirectory,"(?:\t{3,7}       (?!db|dd)[a-zA-Z]{2,6} {1,})")) # cleaning and pulling instructions

        pandasSeriesTest = pd.Series(instructionsForThisFile).value_counts().index, pd.Series(instructionsForThisFile).value_counts().values # Counting each instruction up   
        for i in range(0, len(pandasSeriesTest[0])):
            dataFrame.loc[file,pandasSeriesTest[0][i]] = pandasSeriesTest[1][i]  #0 = instruction and 1 = count columns ||| Second value is index within that column
        
        #Optional cleaning options for my DF to merge dupe columns and group them up
        dataFrame = dataFrame.groupby(axis=1, level=0).sum() # Merges dupe columns
        #dataFrame = dataFrame.loc[:, (dataFrame != 0).any(axis=0)] # Removes columns with no values
    return dataFrame

## Pulling the files from the dataset into the class folders

In [87]:
#moveFilesToClassFolders(listFilesInDirectory("/home/eddy/machine-learning/data/dataset-subsetFullInitialSubset/*"),pd.read_csv("/home/eddy/machine-learning/data/trainLabels.csv"))
#moveFilesToClassFolders(listFilesInDirectory("/home/eddy/machine-learning/data/dataset-subsetBackup/*"),pd.read_csv("/home/eddy/machine-learning/data/trainLabels.csv"))
print(len(glob.glob("/home/eddy/machine-learning/data/dataset-subset/class-1/*")))

187


In [88]:
testDirList = ["/home/eddy/machine-learning/data/dataset-subset/class-1/*","/home/eddy/machine-learning/data/dataset-subset/class-2/*","/home/eddy/machine-learning/data/dataset-subset/class-3/*","/home/eddy/machine-learning/data/dataset-subset/class-4/*","/home/eddy/machine-learning/data/dataset-subset/class-5/*","/home/eddy/machine-learning/data/dataset-subset/class-6/*","/home/eddy/machine-learning/data/dataset-subset/class-7/*","/home/eddy/machine-learning/data/dataset-subset/class-8/*","/home/eddy/machine-learning/data/dataset-subset/class-9/*"]
for directory in testDirList:
    fileList = glob.glob(directory)
    print(len(fileList))
    for i in range(0,len(fileList)):
        if(i >= 100):
            os.remove(fileList[i])
    print(len(fileList))

187
187
196
196
194
194
178
178
84
84
180
180
170
170
185
185
179
179


## Creating the Pandas DataFrame for the malware classes

In [89]:
instructionList = fileNewlineIntoList("/home/eddy/machine-learning/instructionListComplete.txt")
instructionList = [instruction.lower() for instruction in instructionList] # Making all instructions lowercase

dataframeClassOne = generateClassDataFrame(instructionList,cleanFileNameList(listFilesInDirectory("/home/eddy/machine-learning/data/dataset-subset/class-1/*.asm*"),1))
dataframeClassOne = populateMalwareDataframe("/home/eddy/machine-learning/data/dataset-subset/class-1/*.asm",instructionList)
dataframeClassOne.to_csv("/home/eddy/machine-learning/data/datasetClassOne.csv")
#dataframeClassOne = dataframeClassOne.div(dataframeClassOne.sum(axis=1), axis=0)
dataframeClassOne = zeroOutDataframe(dataframeClassOne)
dataframeClassOne.loc[~(dataframeClassOne==0).all(axis=1)]
dataframeClassOne.insert(0,"class",1)
#dataframeClassOneNormalised = dataframeClassOne.div(dataframeClassOne.sum(axis=1), axis=0)
#dataframeClassOneNormalised = zeroOutDataframe(dataframeClassOneNormalised)
#dataframeClassOneNormalised.loc[~(dataframeClassOneNormalised==0).all(axis=1)]
#dataframeClassOneNormalised.insert(0,"class",1)


dataframeClassTwo = generateClassDataFrame(instructionList,cleanFileNameList(listFilesInDirectory("/home/eddy/machine-learning/data/dataset-subset/class-2/*.asm*"),2))
dataframeClassTwo = populateMalwareDataframe("/home/eddy/machine-learning/data/dataset-subset/class-2/*.asm",instructionList)
dataframeClassTwo.to_csv("/home/eddy/machine-learning/data/datasetClassTwo.csv")
#dataframeClassTwo = dataframeClassTwo.div(dataframeClassTwo.sum(axis=1), axis=0)
dataframeClassTwo = zeroOutDataframe(dataframeClassTwo)
dataframeClassTwo.loc[~(dataframeClassTwo==0).all(axis=1)]
dataframeClassTwo.insert(0,"class",2)
#dataframeClassTwoNormalised = dataframeClassTwo.div(dataframeClassTwo.sum(axis=1), axis=0)
#dataframeClassTwoNormalised = zeroOutDataframe(dataframeClassTwoNormalised)
#dataframeClassTwoNormalised.loc[~(dataframeClassTwoNormalised==0).all(axis=1)]
#dataframeClassTwoNormalised.insert(0,"class",2)

dataframeClassThree = generateClassDataFrame(instructionList,cleanFileNameList(listFilesInDirectory("/home/eddy/machine-learning/data/dataset-subset/class-3/*.asm*"),3))
dataframeClassThree = populateMalwareDataframe("/home/eddy/machine-learning/data/dataset-subset/class-3/*.asm",instructionList)
dataframeClassThree.to_csv("/home/eddy/machine-learning/data/datasetClassThree.csv")
#dataframeClassThree = dataframeClassThree.div(dataframeClassThree.sum(axis=1), axis=0)
dataframeClassThree = zeroOutDataframe(dataframeClassThree)
dataframeClassThree.loc[~(dataframeClassThree==0).all(axis=1)]
dataframeClassThree.insert(0,"class",3)
#dataframeClassThreeNormalised = dataframeClassThree.div(dataframeClassThree.sum(axis=1), axis=0)
#dataframeClassThreeNormalised = zeroOutDataframe(dataframeClassThreeNormalised)
#dataframeClassThreeNormalised.loc[~(dataframeClassThreeNormalised==0).all(axis=1)]
#dataframeClassThreeNormalised.insert(0,"class",3)

dataframeClassFour = generateClassDataFrame(instructionList,cleanFileNameList(listFilesInDirectory("/home/eddy/machine-learning/data/dataset-subset/class-4/*.asm*"),4))
dataframeClassFour = populateMalwareDataframe("/home/eddy/machine-learning/data/dataset-subset/class-4/*.asm",instructionList)
dataframeClassFour.to_csv("/home/eddy/machine-learning/data/datasetClassFour.csv")
#dataframeClassFour = dataframeClassFour.div(dataframeClassFour.sum(axis=1), axis=0)
dataframeClassFour= zeroOutDataframe(dataframeClassFour)
dataframeClassFour.loc[~(dataframeClassFour==0).all(axis=1)]
dataframeClassFour.insert(0,"class",4)
#dataframeClassFourNormalised = dataframeClassFour.div(dataframeClassFour.sum(axis=1), axis=0)
#dataframeClassFourNormalised= zeroOutDataframe(dataframeClassFourNormalised)
#dataframeClassFourNormalised.loc[~(dataframeClassFourNormalised==0).all(axis=1)]
#dataframeClassFourNormalised.insert(0,"class",4)

dataframeClassFive = generateClassDataFrame(instructionList,cleanFileNameList(listFilesInDirectory("/home/eddy/machine-learning/data/dataset-subset/class-5/*.asm*"),5))
dataframeClassFive = populateMalwareDataframe("/home/eddy/machine-learning/data/dataset-subset/class-5/*.asm",instructionList)
dataframeClassFive.to_csv("/home/eddy/machine-learning/data/datasetClassFive.csv")
#dataframeClassFive = dataframeClassFive.div(dataframeClassFive.sum(axis=1), axis=0)
dataframeClassFive = zeroOutDataframe(dataframeClassFive)
dataframeClassFive.loc[~(dataframeClassFive==0).all(axis=1)]
dataframeClassFive.insert(0,"class",5)
#dataframeClassFiveNormalised = zeroOutDataframe(dataframeClassFiveNormalised)
#dataframeClassFiveNormalised.loc[~(dataframeClassFiveNormalised==0).all(axis=1)]
#dataframeClassFiveNormalised.to_csv("/home/eddy/machine-learning/data/datasetClassFiveNormalised.csv")
#dataframeClassFiveNormalised.insert(0,"class",5)

dataframeClassSix = generateClassDataFrame(instructionList,cleanFileNameList(listFilesInDirectory("/home/eddy/machine-learning/data/dataset-subset/class-6/*.asm*"),6))
dataframeClassSix = populateMalwareDataframe("/home/eddy/machine-learning/data/dataset-subset/class-6/*.asm",instructionList)
dataframeClassSix.to_csv("/home/eddy/machine-learning/data/datasetClassSix.csv")
#dataframeClassSix = dataframeClassSix.div(dataframeClassSix.sum(axis=1), axis=0)
dataframeClassSix = zeroOutDataframe(dataframeClassSix)
dataframeClassSix.loc[~(dataframeClassSix==0).all(axis=1)]
dataframeClassSix.insert(0,"class",6)
#dataframeClassSixNormalised = dataframeClassSix.div(dataframeClassSix.sum(axis=1), axis=0)
#dataframeClassSixNormalised = zeroOutDataframe(dataframeClassSixNormalised)
#dataframeClassSixNormalised.loc[~(dataframeClassSixNormalised==0).all(axis=1)]
#dataframeClassSixNormalised.insert(0,"class",6)

dataframeClassSeven = generateClassDataFrame(instructionList,cleanFileNameList(listFilesInDirectory("/home/eddy/machine-learning/data/dataset-subset/class-7/*.asm*"),7))
dataframeClassSeven = populateMalwareDataframe("/home/eddy/machine-learning/data/dataset-subset/class-7/*.asm",instructionList)
dataframeClassSeven.to_csv("/home/eddy/machine-learning/data/datasetClassSeven.csv")
#dataframeClassSeven = dataframeClassSeven.div(dataframeClassSeven.sum(axis=1), axis=0)
dataframeClassSeven = zeroOutDataframe(dataframeClassSeven)
dataframeClassSeven.loc[~(dataframeClassSeven==0).all(axis=1)]
dataframeClassSeven.insert(0,"class",7)
#dataframeClassSevenNormalised = dataframeClassSeven.div(dataframeClassSeven.sum(axis=1), axis=0)
#dataframeClassSevenNormalised = zeroOutDataframe(dataframeClassSevenNormalised)
#dataframeClassSevenNormalised.loc[~(dataframeClassSevenNormalised==0).all(axis=1)]
#dataframeClassSevenNormalised.insert(0,"class",7)

dataframeClassEight = generateClassDataFrame(instructionList,cleanFileNameList(listFilesInDirectory("/home/eddy/machine-learning/data/dataset-subset/class-8/*.asm*"),8))
dataframeClassEight = populateMalwareDataframe("/home/eddy/machine-learning/data/dataset-subset/class-8/*.asm",instructionList)
dataframeClassEight.to_csv("/home/eddy/machine-learning/data/datasetClassEight.csv")
#dataframeClassEight = dataframeClassEight.div(dataframeClassEight.sum(axis=1), axis=0)
dataframeClassEight = zeroOutDataframe(dataframeClassEight)
dataframeClassEight.loc[~(dataframeClassEight==0).all(axis=1)]
dataframeClassEight.insert(0,"class",8)
#dataframeClassEightNormalised = dataframeClassEight.div(dataframeClassEight.sum(axis=1), axis=0)
#dataframeClassEightNormalised = zeroOutDataframe(dataframeClassEightNormalised)
#dataframeClassEightNormalised.loc[~(dataframeClassEightNormalised==0).all(axis=1)]
#dataframeClassEightNormalised.insert(0,"class",8)

dataframeClassNine = generateClassDataFrame(instructionList,cleanFileNameList(listFilesInDirectory("/home/eddy/machine-learning/data/dataset-subset/class-9/*.asm*"),9))
dataframeClassNine = populateMalwareDataframe("/home/eddy/machine-learning/data/dataset-subset/class-9/*.asm",instructionList)
dataframeClassNine.to_csv("/home/eddy/machine-learning/data/datasetClassNine.csv")
#dataframeClassNine = dataframeClassNine.div(dataframeClassNine.sum(axis=1), axis=0)
dataframeClassNine = zeroOutDataframe(dataframeClassNine)
dataframeClassNine.loc[~(dataframeClassNine==0).all(axis=1)]
dataframeClassNine.insert(0,"class",9)
#dataframeClassNineNormalised = dataframeClassNine.div(dataframeClassNine.sum(axis=1), axis=0)
#dataframeClassNineNormalised = zeroOutDataframe(dataframeClassNineNormalised)
#dataframeClassNineNormalised.loc[~(dataframeClassNineNormalised==0).all(axis=1)]
#dataframeClassNineNormalised.insert(0,"class",9)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 657871: invalid continuation byte

In [ ]:
#dataframesNormalisedList = [dataframeClassOneNormalised,dataframeClassTwoNormalised,dataframeClassThreeNormalised,dataframeClassFourNormalised,dataframeClassFiveNormalised,dataframeClassSixNormalised,dataframeClassSevenNormalised,dataframeClassEightNormalised,dataframeClassNineNormalised]
dataframesList = [dataframeClassOne,dataframeClassTwo,dataframeClassThree,dataframeClassFour,dataframeClassFive,dataframeClassSix,dataframeClassSeven,dataframeClassEight,dataframeClassNine]
finalDF = pd.concat(dataframesList).drop_duplicates()
#finalDF = finalDF.drop(["assume","align"],axis=1) # Error, seems like they're gone already
finalDF = zeroOutDataframe(finalDF)
finalDF.loc[~(finalDF==0).all(axis=1)]
finalDF = finalDF.loc[:, (finalDF != 0).any(axis=0)] # Removes columns with no values
finalDF = finalDF.sample(frac=1)

## Model Setup

In [ ]:
#Importing sklearn and getting basic info about my training set
#Merge all data into one DF with labels intact

from sklearn import svm
from sklearn.model_selection import train_test_split

finalDF.info()

## Splitting the data into train+test sets

In [ ]:
#Dividing up the dataset into train, validate and test sets
trainDF, testAndValidDF = train_test_split(finalDF, test_size=0.4)
testDF, validDF = train_test_split(testAndValidDF, test_size=0.5)

print(f"Training Dataset rows and columns: {trainDF.shape}")
print(f"Test Dataset rows and columns: {testDF.shape}")
print(f"Validation Dataset rows and columns: {validDF.shape}")

## Training Stats

In [ ]:
trainStats = trainDF.describe()
trainStats.pop("class")
#not doing sns here

In [ ]:
#Creating training stats based on the trainDF dataset
trainStats = trainDF.describe()
trainStats.pop("class")
trainStats = trainStats.transpose()
trainStats.to_csv("/home/eddy/traindata.csv")

In [ ]:
#Creating Labels
trainLabels = trainDF.pop("class")
testLabels = testDF.pop("class")
validLabels = validDF.pop("class")

print(len(trainLabels))
print(len(testLabels))
print(len(validLabels))

## Data Normalisation/Scaling

In [ ]:
def normaliseData(data):
    #return (data -trainStats["mean"]) / trainStats['std'] #Works fine, experimenting with the OTHER
    return data
    #return data.div(data.sum(axis=1), axis=0)

normalisedTrainDF = normaliseData(trainDF)
normalisedTestDF = normaliseData(testDF)
normalisedValidDF = normaliseData(validDF)

normalisedTrainDF = normalisedTrainDF.replace(np.nan,0)
normalisedTestDF = normalisedTestDF.replace(np.nan,0)
normalisedValidDF = normalisedValidDF.replace(np.nan,0)
normalisedTrainDF.head(10)

## Training the model

In [ ]:
#Create a svm Classifier
model = svm.SVC(C = 1.5,
                kernel='linear')
modelPoly = svm.SVC(C = 1.5,
                   kernel='poly')
modelRBF = svm.SVC(C = 1.5,
                   kernel='rbf')
modelSig = svm.SVC(C = 1.5,
                   kernel='sigmoid')

#Train the model using the training sets
model.fit(normalisedTrainDF, trainLabels)
#Predict the response for test dataset
y_pred = model.predict(normalisedTrainDF)

modelPoly.fit(normalisedTrainDF, trainLabels)
y_predPoly = modelPoly.predict(normalisedTrainDF)

modelRBF.fit(normalisedTrainDF, trainLabels)
y_predRBF = modelRBF.predict(normalisedTrainDF)

modelSig.fit(normalisedTrainDF, trainLabels)
y_predSig = modelSig.predict(normalisedTrainDF)

In [ ]:
exampleBatch = normalisedTestDF[:10]

exampleResult = model.predict(exampleBatch)

print(pd.Series(list(exampleBatch.index),index=exampleResult).to_string())
print(f"Predicted values: {exampleResult}")

## Checking how training went

In [ ]:
from sklearn import metrics

#print(normalisedTrainDF.shape)
normalisedTrainDF = normalisedTrainDF[np.isfinite(normalisedTrainDF).all(1)]
#print(normalisedTrainDF.shape)

#print(len(y_pred))
#y_pred = model.predict(normalisedTrainDF)
print("Linear Train Accuracy: ",metrics.accuracy_score(trainLabels,y_pred))
print("Poly Train Accuracy: ",metrics.accuracy_score(trainLabels,y_predPoly))
print("RBF Train Accuracy: ",metrics.accuracy_score(trainLabels,y_predRBF))
print("Sigmoid Train Accuracy: ",metrics.accuracy_score(trainLabels,y_predSig))

In [ ]:
#newNormalisedValidDF = zeroOutDataframe(normalisedValidDF)
#newNormalisedValidDF = newNormalisedValidDF.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

y_pred = model.predict(normalisedValidDF)
print("Linear Valid Accuracy: ",metrics.accuracy_score(validLabels,y_pred))

In [ ]:
#newNormalisedTestDF = zeroOutDataframe(normalisedTestDF)
#newNormalisedTestDF = newNormalisedTestDF.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

y_pred = model.predict(normalisedTestDF)
print("Test Accuracy: ",metrics.accuracy_score(testLabels,y_pred))

In [ ]:
ax = plt.subplot()
predictResults = model.predict(normalisedTestDF)
cm = confusion_matrix(predictResults,predictResults)

ax.set_xlabel("Predicted Labels")
ax.set_ylabel("True Labels")
ax.set_title("Confusion Matrix - Linear")
ax.set_xticks([1,2,3,4,5,6,7,8,9])

sns.heatmap(cm, annot=True, ax=ax, yticklabels=["1","2","3","4","5","6","7","8","9"], xticklabels=["1","2","3","4","5","6","7","8","9"]); #Semicolon removes the annoying text above the graph

In [ ]:
print(classification_report(testLabels,y_pred))

In [ ]:
permutationImportance = permutation_importance(model, normalisedTrainDF, trainLabels)
featuresList = np.array(list(normalisedTrainDF.columns))
sortedIDX = permutationImportance.importances_mean.argsort()
mostImportantIndexesPermutation = [list(permutationImportance.importances_mean[sortedIDX]).index(i) for i in heapq.nlargest(30, permutationImportance.importances_mean[sortedIDX])]

### Showing the largest features
newFeaturesList = []
newPermutationImportanceList = []

for i in mostImportantIndexesPermutation[::-1]:
    newFeaturesList.append(featuresList[sortedIDX][i])
    newPermutationImportanceList.append(permutationImportance.importances_mean[sortedIDX][i])

occurancesQuantity={}
for i in newFeaturesList[::-1]:
    occurancesQuantity.update({i:str(int(finalDF[i].mean()))})


from sklearn import preprocessing
plt.subplot(1, 2, 1)
plt.barh(newFeaturesList, newPermutationImportanceList);
plt.xlabel("Permutation Importance/Feature");
plt.margins(x=0)
plt.xticks([0,0.1,0.2,0.3,0.4,0.5],["0","0.2","0.4","0.6","0.8","1"])

plt.subplot(1, 2, 2)
plt.barh(list(occurancesQuantity.keys())[::-1], preprocessing.minmax_scale(list(occurancesQuantity.values())[::-1],feature_range=(0,0.5)));
plt.xlabel("Mean Relative occurances/Feature");
plt.xticks([0,0.1,0.2,0.3,0.4,0.5],["0","0.2","0.4","0.6","0.8","1"])
plt.margins(x=0)
plt.tight_layout()


In [ ]:
print(classification_report(testLabels,y_pred,output_dict=True))